<a href="https://colab.research.google.com/github/maxencepenaud/Economic-Modelling-Session-15-final-/blob/main/Final_Modeling_Session_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.calibration import CalibratedClassifierCV
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown, HTML

# This function simulates drawing a card from a deck: It randomly selects a card from a standard deck of cards. Face cards (Jack, Queen, King) are valued at 10 and Aces are valued at 11 for simplicity. Number cards are returned as their integer values.
def draw_card():
    card = np.random.choice(['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A'])
    if card in ['J', 'Q', 'K']:
        return 10
    elif card == 'A':
        return 11  # Simplification for this example
    else:
        return int(card)

# Function to calculate the best hand value (considering Aces as 1 or 11).This function calculates the best possible hand value: If the total value exceeds 21 and there are Aces we deduct 10 and aces become 1.
def hand_value(hand):
    value = sum(hand)
    num_aces = hand.count(11)
    while value > 21 and num_aces:
        value -= 10
        num_aces -= 1
    return value

# Simulate a Blackjack game dataset: This function simulates Blackjack games to create a dataset. It initializes an empty list, then for each game, draws two cards for the player and dealer, calculates their hand values, and checks for an immediate win or tie if there's a Blackjack. If not, the player may draw more cards with a 50% chance, and the dealer draws until their hand value is at least 17. The game result (win, lose, tie) is recorded along with the player’s and dealer’s hand values and the player's card count. The data is returned as a DataFrame.
def simulate_blackjack_games(num_games=100):
    data = []
    for _ in range(num_games):
        player_hand = [draw_card(), draw_card()]
        dealer_hand = [draw_card(), draw_card()]

        player_value = hand_value(player_hand)
        dealer_value = hand_value(dealer_hand)

        if player_value == 21:
            result = 'win' if dealer_value != 21 else 'tie'
        elif dealer_value == 21:
            result = 'lose'
        else:
            while player_value < 21 and np.random.rand() > 0.5:  # Random hit/stay
                player_hand.append(draw_card())
                player_value = hand_value(player_hand)

            while dealer_value < 17:
                dealer_hand.append(draw_card())
                dealer_value = hand_value(dealer_hand)

            if player_value > 21:
                result = 'lose'
            elif dealer_value > 21 or player_value > dealer_value:
                result = 'win'
            elif player_value < dealer_value:
                result = 'lose'
            else:
                result = 'tie'

        data.append({
            'player_value': player_value,
            'dealer_value': dealer_value,
            'player_card_count': len(player_hand),  # New feature
            'result': result
        })

    return pd.DataFrame(data)

# Generate the dataset
blackjack_data = simulate_blackjack_games(100)

# Encode the target variable
blackjack_data['result'] = blackjack_data['result'].map({'win': 1, 'lose': -1, 'tie': 0})

# Separate features (X) and target variable (y)
X = blackjack_data[['player_value', 'dealer_value', 'player_card_count']]  # Updated features
y = blackjack_data['result']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate and train the RandomForest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Calibrate probabilities using Isotonic Regression
calibrated_rf = CalibratedClassifierCV(estimator=rf_model, method='isotonic', cv='prefit')  # Updated to 'estimator'
calibrated_rf.fit(X_test, y_test)  # Use the test set for calibration

# Make predictions on the testing set using calibrated model
y_pred = calibrated_rf.predict(X_test)

# Evaluate the model performance
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Function to plot the confusion matrix
def plot_confusion_matrix(cm):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
                xticklabels=['Lose', 'Tie', 'Win'], yticklabels=['Lose', 'Tie', 'Win'])
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.show()

# Function to display outputs using Markdown for better formatting
def display_markdown(text):
    display(Markdown(text))

# Function to predict probabilities using calibrated model
def predict_probabilities(player_value, dealer_value, player_card_count):
    features = pd.DataFrame({
        'player_value': [player_value],
        'dealer_value': [dealer_value],
        'player_card_count': [player_card_count]
    })
    probs = calibrated_rf.predict_proba(features)[0]
    return {'lose': probs[0], 'tie': probs[1], 'win': probs[2]}

# Function to play the game interactively with enhanced stats
def play_blackjack():
    player_hand = [draw_card(), draw_card()]
    dealer_hand = [draw_card(), draw_card()]

    while True:
        player_value = hand_value(player_hand)
        dealer_value = hand_value(dealer_hand)
        player_card_count = len(player_hand)

        display_markdown(f"**Your hand:** {player_hand} (Value: {player_value})")
        display_markdown(f"**Dealer's face-up card:** {dealer_hand[0]}")

        probabilities = predict_probabilities(player_value, dealer_value, player_card_count)
        display_markdown(f"**Probabilities** - Win: `{probabilities['win']:.2f}`, Lose: `{probabilities['lose']:.2f}`, Tie: `{probabilities['tie']:.2f}`")

        # Suggest action based on probabilities
        if probabilities['win'] > probabilities['lose'] and probabilities['win'] > probabilities['tie']:
            display_markdown("**Tip:** Consider taking a risk!")
        elif probabilities['tie'] > probabilities['lose']:
            display_markdown("**Tip:** Playing safe could be beneficial.")
        else:
            display_markdown("**Tip:** Caution advised, consider staying.")

        if player_value >= 21:
            break

        choice = input("Do you want to hit or stay? (hit/stay): ").strip().lower()

        if choice == 'hit':
            player_hand.append(draw_card())
            player_value = hand_value(player_hand)
            player_card_count = len(player_hand)
            if player_value > 21:
                display_markdown(f"**You busted with a hand value of {player_value}. You lose!**")
                return
            # Update probabilities after the hit
            probabilities = predict_probabilities(player_value, dealer_value, player_card_count)
            display_markdown(f"**Updated Probabilities** after hit - Win: `{probabilities['win']:.2f}`, Lose: `{probabilities['lose']:.2f}`, Tie: `{probabilities['tie']:.2f}`")
        elif choice == 'stay':
            break

    while dealer_value < 17:
        dealer_hand.append(draw_card())
        dealer_value = hand_value(dealer_hand)

    display_markdown(f"**Dealer's hand:** {dealer_hand} (Value: {dealer_value})")

    if dealer_value > 21 or player_value > dealer_value:
        display_markdown("**You win!**")
    elif player_value < dealer_value:
        display_markdown("**You lose!**")
    else:
        display_markdown("**It's a tie!**")

# Example usage
play_blackjack()

**Your hand:** [10, 6] (Value: 16)

**Dealer's face-up card:** 10

**Probabilities** - Win: `0.30`, Lose: `0.70`, Tie: `0.00`

**Tip:** Caution advised, consider staying.

Do you want to hit or stay? (hit/stay): hit


...

**Dealer's face-up card:** 10

**Probabilities** - Win: `1.00`, Lose: `0.00`, Tie: `0.00`

**Tip:** Consider taking a risk!

**Dealer's hand:** [10, 2, 10] (Value: 22)

**You win!**